<a href="https://colab.research.google.com/github/joshpc/CanadaEnergy2020/blob/main/Climate_Research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Canadian Energy Solutions

This is a simple notebook that allows us to explore some of the underlying problems in the Canadian energy space. The objective is to identify sectors that need assistance, and potentially identify companies that can fill those gaps.

If we were to simplify the Canadian energy problem, we can say that we need to reach a scenario where two simple constraints are met:

`supply > demand && net_emissions <= 0`

In reality, this is a much more complex equation since if we were to sum the entirety of Canada's production, and subtract demand, we would be glossing over regional differences and distances.

For the sake of this document, we will start by assuming two things:
- A province must be fully sufficient. That is to say that it must not depend on neighbouring provinces to provide electricity.
- Emissions across the country must be net. One province may pollute, as long as neighbouring provinces can assist with the reduction (it's all one atmosphere.)

In [ ]:
%load_ext google.colab.data_table
%matplotlib inline

# import csv
import ssl

# We're disabling SSL Cert Verification for these data examples
ssl._create_default_https_context = ssl._create_unverified_context

# from google.colab import drive

import numpy as np
import pandas as pd

from google.colab import data_table

from matplotlib import pyplot as plt

import plotly.graph_objects as go

In [ ]:
# Reset all data
butane_data = None
coal_data = None
crude_oil_data = None
electricity_data = None
ethane_data = None
natural_gas_data = None
pentanes_data = None
propane_data = None
end_use_demand_data = None

In [ ]:
# Let's import our data

# This uses Canada's Energy Futures 2020 Dataset. This dataset looks far into the future and gives us a good idea of whether
# we're heading in the right direction or not. This isn't necessarily a 100% accurate, as some data sources are limited but it's
# a good start.
#
# >  The Energy Futures series explores how possible energy futures might unfold for Canadians over the long term.
# >  Canada’s Energy Future 2020: Energy Supply and Demand Projections to 2050 (EF2020) is our latest long-term energy outlook.
# >  It is the first outlook in the series to provide projections to 2050.
# >  It covers all energy commodities, and all provinces and territories.
# >  We use economic and energy models to develop this outlook. We also make assumptions about technology,
# >  energy and climate policies, energy markets, human behaviour and the economy.
#

# https://open.canada.ca/data/en/dataset/bba41250-261a-4f3b-9ce8-db44d9a0f725

# Butane
print("Importing Butane Data...")
butane_data = pd.read_csv("https://www.cer-rec.gc.ca/open/energy/energyfutures2020/butanes-2020.csv")

# Coal
print("Importing Coal Data...")
coal_data = pd.read_csv("https://www.cer-rec.gc.ca/open/energy/energyfutures2020/coal-2020.csv")

# Crude Oil
print("Importing Crude Oil Data...")
crude_oil_production_data = pd.read_csv("https://www.cer-rec.gc.ca/open/energy/energyfutures2020/crude-oil-production-2020.csv")

# Electricity Generation
print("Importing Electricity Generation Data...")
electricity_data = pd.read_csv("https://www.cer-rec.gc.ca/open/energy/energyfutures2020/electricity-generation-2020.csv")

# Ethane
print("Importing Ethane Data...")
ethane_data = pd.read_csv("https://www.cer-rec.gc.ca/open/energy/energyfutures2020/ethane-2020.csv")

# Natural Gas Production
print("Importing Natural Gas Data...")
natural_gas_data = pd.read_csv("https://www.cer-rec.gc.ca/open/energy/energyfutures2020/natural-gas-production-2020.csv")

# Pentanes
print("Importing Pentanes Data...")
pentanes_data = pd.read_csv("https://www.cer-rec.gc.ca/open/energy/energyfutures2020/pentanes-2020.csv")

# Propane
print("Importing Propane Data...")
propane_data = pd.read_csv("https://www.cer-rec.gc.ca/open/energy/energyfutures2020/propane-2020.csv")

# Primary Demand
print("Importing Primary Demand...")
primary_demand_data = pd.read_csv("https://www.cer-rec.gc.ca/open/energy/energyfutures2020/primary-energy-demand-2020.csv")

# End Use Demand
print("Importing End Use Demand Data...")
end_use_demand_data = pd.read_csv("https://www.cer-rec.gc.ca/open/energy/energyfutures2020/end-use-demand-2020.csv")

# Electric power generation, monthly generation by type of electricity
# Source: https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=2510001501
#
# https://www150.statcan.gc.ca/t1/tbl1/#?pid=25100015&file=2510001501-eng.csv

print("Done!")

Importing Butane Data...
Importing Coal Data...
Importing Crude Oil Data...
Importing Electricity Generation Data...
Importing Ethane Data...
Importing Natural Gas Data...
Importing Pentanes Data...
Importing Propane Data...
Importing Primary Demand...
Importing End Use Demand Data...
Done!


In [ ]:
# Data setup
layer_1_production_and_imports = {}
layer_2_primary_energy = {}
layer_3_secondary_energy = {}
layer_4_industry = {}
layer_5_industry_end_use = {}

In [ ]:
# Helper Methods
# Conversion Data: https://apps.cer-rec.gc.ca/Conversion/conversion-tables.aspx?GoCTemplateCulture=en-CA

gj_to_pj = 1000000

def gigajoule_to_quad(gj):
  return -1

# Butane, in Cubic Metres
def butane_to_pj(butane):
  return butane * 28.62 / gj_to_pj

# Coal, in tons
# Coal has four different "types" each resulting in different energy densities.
#   Anthracite: 27.70 GJ
#   Bituminous: 27.60 GJ
#   Lignite: 14.40 GJ
#   Subbituminous: 18.80 GJ
def coal_to_pj(coal):
  return coal * 27.60 / gj_to_pj

# Crude Oil, in Cubic Metres
#   Pentanes plus	1.0 Cubic metres (m³)	35.17 Gigajoules (GJ)
#   Light	1.0 Cubic metres (m³)	38.51 Gigajoules (GJ)
#   Heavy	1.0 Cubic metres (m³)	40.90 Gigajoules (GJ)
#   Synthetic crude oil	1.0 Cubic metres (m³)	39.40 Gigajoules (GJ)
#   Bitumen	1.0 Cubic metres (m³)	42.80 Gigajoules (GJ)
def crude_oil_to_pj(crude_oil):
  #TODO: Differentiate
  return crude_oil * 40.90 / gj_to_pj

# Ethane, in Cubic Metres
def ethane_to_pj(ethane):
  return ethane * 18.36 / gj_to_pj

# Natural Gas, in Cubic Metres
def natural_gas_to_pj(natural_gas):
  return natural_gas * 0.0373 / gj_to_pj

def pentanes_to_pj(pentanes):
  return pentanes * 35.17 / gj_to_pj

def propane_to_pj(propane):
  return propane * 25.53 / gj_to_pj

# Using https://www.cesarnet.ca/visualization/sankey-diagrams-canadas-energy-systems 's conversion,
# > The primary heat energy generated by uranium depends on the reactor technology being used, and is much higher in reactors that use enriched uranium than in the CANDU natural uranium reactors used in Canada. The typical heat rate for uranium in a CANDU is 7700 MW-days(thermal)/tonne U, which converts to 0.665 PJ/tonne U, and this is the the conversion factor used in this Sankey diagram for estimating the primary energy content of uranium.
def uranium_to_pj(uranium):
  return uranium * 0.665

In [ ]:
def extract_butane_data(layer_1, data, year):
  butane_data = data[(data["Year"] == year) & (data["Unit"] == "Thousand Cubic Metres per day")]

  # Butane data is "per day". We assume 365 day averages.
  layer_1["butane_imports"] = butane_to_pj(1000 * 365 * butane_data[(butane_data["Variable"] == "Imports to AB from U.S.")]["Value"].sum())
  layer_1["butane_production"] = butane_to_pj(1000 * 365 * butane_data[(butane_data["Variable"].isin(["Production from Gas Processing", "Production from Oil Sands off-gas"]))]["Value"].sum())
  layer_1["butane_exports"] = butane_to_pj(1000 * 365 * butane_data[(butane_data["Variable"] == "Projected Exports")]["Value"].sum())

In [ ]:
def extract_coal_data(layer_1, data, year):
  coal_data = data[data["Year"] == year]

  # These values are all in kilotons
  layer_1["coal_imports"] = coal_to_pj(1000 * coal_data[(coal_data["Variable"] == "Total") & (coal_data["Type"] == "Imports")].iloc[0]["Value"])
  layer_1["coal_production"] = coal_to_pj(1000 * coal_data[(coal_data["Variable"] == "Total") & (coal_data["Type"] == "Production")].iloc[0]["Value"])
  layer_1["coal_exports"] = coal_to_pj(1000 * coal_data[(coal_data["Variable"] == "Total") & (coal_data["Type"] == "Exports")].iloc[0]["Value"])

  #TODO: Use Domestic Demand for other layers

In [ ]:
def extract_crude_oil_data(layer_1, data, year):
  crude_oil_data = data[(data["Year"] == year) & (data["Region"] == "Canada") & (data["Unit"] == "Thousand Cubic Metres per day")]

  # Crude oil extract data is in a daily format
  layer_1["crude_oil_imports"] = -1
  layer_1["crude_oil_production"] = crude_oil_to_pj(1000 * 365 * crude_oil_data[crude_oil_data["Variable"] == "Total"].iloc[0]["value"])
  layer_1["crude_oil_exports"] = -1

In [ ]:
def extract_electricity_data(layer_2, data, year):
  electricity_data = data[(data["Year"] == year) & (data["Region"] == "Canada")]

  #TODO: Convert units
  layer_2["hydro_wave_tidal"] = electricity_data[(electricity_data["Variable"] == "Hydro / Wave / Tidal")].iloc[0]["Value"]
  layer_2["natural_gas"] = electricity_data[(electricity_data["Variable"] == "Natural Gas")].iloc[0]["Value"]
  layer_2["oil"] = electricity_data[(electricity_data["Variable"] == "Oil")].iloc[0]["Value"]
  layer_2["biomass_geothermal"] = electricity_data[(electricity_data["Variable"] == "Biomass / Geothermal")].iloc[0]["Value"]
  #TODO: Can this not be split?
  layer_2["coal_and_coke"] = electricity_data[(electricity_data["Variable"] == "Coal & Coke")].iloc[0]["Value"]
  layer_2["uranium"] = electricity_data[(electricity_data["Variable"] == "Uranium")].iloc[0]["Value"]
  layer_2["solar"] = electricity_data[(electricity_data["Variable"] == "Solar")].iloc[0]["Value"]
  layer_2["wind"] = electricity_data[(electricity_data["Variable"] == "Wind")].iloc[0]["Value"]

In [ ]:
def extract_ethane_data(layer_1, data, year):
  ethane_data = data[(data["Year"] == year) & (data["Unit"] == "Thousand Cubic Metres per day")]

  layer_1["ethane_imports"] = ethane_to_pj(1000 * 365 * ethane_data[ethane_data["Variable"].isin(["Imports to AB from U.S.", "Imports to ON from U.S."])]["Value"].sum())
  layer_1["ethane_production"] = ethane_to_pj(1000 * 365 * ethane_data[ethane_data["Variable"].isin(["Production from Gas Processing", "Production from Oil Sands off-gas"])]["Value"].sum())
  layer_1["ethane_exports"] = ethane_to_pj(1000 * 365 * ethane_data[ethane_data["Variable"] == "Exports"].iloc[0]["Value"])

In [ ]:
def extract_natural_gas_data(layer_1, data, year):
  #TODO: Get import/export
  natural_gas_data = data[(data["Year"] == year) & (data["Region"] == "Canada") & (data["Unit"] == "Million Cubic Metres per day")]

  layer_1["natural_gas_imports"] = -1
  layer_1["natural_gas_production"] = natural_gas_to_pj(1000000 * 365 * natural_gas_data.iloc[0]["Value"])
  layer_1["natural_gas_exports"] = -1

In [ ]:
def extract_pentanes_data(layer_1, data, year):  
  pentanes_data = data[(data["Year"] == year) & (data["Unit"] == "Thousand Cubic Metres per day")]

  layer_1["pentane_imports"] = pentanes_to_pj(1000 * 365 * pentanes_data[pentanes_data["Variable"] == "Net Imports"].iloc[0]["Value"])
  layer_1["pentane_production"] = pentanes_to_pj(1000 * 365 * pentanes_data[pentanes_data["Variable"].isin(["Produciton from Refineries", "Production from Gas Processing - Pentanes Plus", "Production from Gas Wells - Liquid Condensate"])]["Value"].sum())
  layer_1["pentane_exports"] = pentanes_to_pj(1000 * 365 * pentanes_data[pentanes_data["Variable"] == "Net Exports"].iloc[0]["Value"])

In [ ]:
def extract_propane_data(layer_1, data, year):
  propane_data = data[(data["Year"] == year) & (data["Unit"] == "Thousand Cubic Metres per day")]

  layer_1["propane_imports"] = propane_to_pj(1000 * 365 * propane_data[propane_data["Variable"] == "Imports"].iloc[0]["Value"])
  layer_1["propane_production"] = propane_to_pj(1000 * 365 * propane_data[propane_data["Variable"].isin(["Production from Gas Processing", "Production from Oil Sands off-gas", "Production from Refineries"])]["Value"].sum())
  layer_1["propane_exports"] = propane_to_pj(1000 * 365 * propane_data[propane_data["Variable"] == "Projected Exports"].iloc[0]["Value"])

In [ ]:
def extract_uranium_data(layer_1):
  # Uranium
  # Source: https://www.world-nuclear.org/information-library/country-profiles/countries-a-f/canada-uranium.aspx
  # We have limited data here, but the totals per year are: (tonnes U3O8)
  # 2019: 8165 
  # 2018: 8256
  # 2017: 15,467
  # 2016: 16,541
  # 2015: 15,709

  # However if we look at Cameco (owner of Canada's Cigar Lake Uranium facility):
  # https://www.cameco.com/invest/financial-information/annual-reports/2020
  # We can see that they produced >50% of 2019's production despite Covid 19 https://www.cameco.com/businesses/uranium-operations/canada/cigar-lake
  # 5,000,000 lbs instead of 9,000,000 -- however this doesn't match Canada's 2019 numbers of 8165 (9m lbs = roughly 5k tons.)
  # Ignoring COVID-19's impact, let's assume they kept the same production.

  layer_1["uranium_imports"] = -1
  layer_1["uranium_production"] = uranium_to_pj(8165) #tons
  layer_1["uranium_exports"] = -1

In [ ]:
def extract_layer_1(layer, year):
  extract_butane_data(layer, butane_data, year)
  extract_coal_data(layer, coal_data, year)
  extract_crude_oil_data(layer, crude_oil_production_data, year)
  extract_ethane_data(layer, ethane_data, year)
  extract_natural_gas_data(layer, natural_gas_data, year)
  extract_pentanes_data(layer, pentanes_data, year)
  extract_propane_data(layer, propane_data, year)
  extract_uranium_data(layer)

def extract_layer_2(layer, year):
  extract_electricity_data(layer, electricity_data, year)

extract_layer_1(layer_1_production_and_imports, 2020)
extract_layer_2(layer_2_primary_energy, 2020)
display(layer_1_production_and_imports)
display(layer_2_primary_energy)

{'butane_exports': 160.0354700761122,
 'butane_imports': 22.081154525028,
 'butane_production': 512.9647136365068,
 'coal_exports': 896.602565244,
 'coal_imports': 162.66573448320003,
 'coal_production': 1245.751417116,
 'crude_oil_exports': -1,
 'crude_oil_imports': -1,
 'crude_oil_production': 11229.3118898779,
 'ethane_exports': 0.0,
 'ethane_imports': 230.099958365724,
 'ethane_production': 477.62991630643666,
 'natural_gas_exports': -1,
 'natural_gas_imports': -1,
 'natural_gas_production': 6062.15880664835,
 'pentane_exports': 0.0,
 'pentane_imports': 395.79014831042747,
 'pentane_production': 1857.7547574999141,
 'propane_exports': 298.14651848710497,
 'propane_imports': 21.6585977511597,
 'propane_production': 872.9869648030601,
 'uranium_exports': -1,
 'uranium_imports': -1,
 'uranium_production': 5429.725}

{'biomass_geothermal': 8197.779,
 'coal_and_coke': 38072.7714,
 'hydro_wave_tidal': 385456.3,
 'natural_gas': 64000.64,
 'oil': 1937.9797,
 'solar': 3321.204427,
 'uranium': 89281.94,
 'wind': 34523.77}

In [ ]:
labels = [
          # Layer 1: Resources (And Layer 2 Exports)
          "Butane Production", "Butane Imports", "Butane Exports",
          "Coal Production", "Coal Imports", "Coal Exports",
          "Crude Oil Production", "Crude Oil Imports", "Crude Oil Exports",
          "Ethane Production", "Ethane Imports", "Ethane Exports",
          "Natural Gas Production", "Natural Gas Imports", "Natural Gas Exports",
          "Pentane Production", "Pentane Imports", "Pentane Exports",
          "Propane Production", "Propane Imports", "Propane Exports",
          "Uranium Production", "Uranium Imports", "Uranium Exports",
          "Hydroelectricity Production", "TODO Renewables Production",

          # Layer 2: Primary Energy
          "Primary Coal, Coke, and Coke Oven Gas",
          "Primary Hydro",
          "Primary Natural Gas",
          "Primary Nuclear",
          "Primary Other Renewables and Landfill Gas",
          "Primary Refined Petroleum Products",

          "TODO Producted & Distributed Energy",

          # Layer 3: "Electricity"
          "Exports",
          "Electricity",
          "Unknown"]
labels_index = {}

for i, label in enumerate(labels):
  labels_index[label] = i

source_nodes = [
  # Production
  0, 0, 1, 1,     # Butane
  3, 3, 4, 4,     # Coal
  6, 6, 7, 7,     # Crude Oil
  9, 9, 10, 10,   # Ethane
  12, 12, 13, 13, # Natural Gas
  15, 15, 16, 16, # Pentane
  18, 18, 19, 19, # Propane
  21, 21, 22, 22, # Uranium
  labels_index["Hydroelectricity Production"], labels_index["TODO Renewables Production"],

  # Primary Energy,
  labels_index["Primary Coal, Coke, and Coke Oven Gas"], labels_index["Primary Hydro"], labels_index["Primary Natural Gas"], labels_index["Primary Nuclear"], labels_index["Primary Other Renewables and Landfill Gas"], labels_index["Primary Refined Petroleum Products"],

  # Exports
  labels_index["Butane Exports"], labels_index["Coal Exports"], labels_index["Crude Oil Exports"], labels_index["Ethane Exports"], labels_index["Natural Gas Exports"], labels_index["Pentane Exports"], labels_index["Propane Exports"], labels_index["Uranium Exports"]    
]

target_nodes = [
  # Production
  2, labels_index["TODO Producted & Distributed Energy"], 2, labels_index["TODO Producted & Distributed Energy"],
  5, labels_index["Primary Coal, Coke, and Coke Oven Gas"], 5, labels_index["Primary Coal, Coke, and Coke Oven Gas"],
  8, labels_index["TODO Producted & Distributed Energy"], 8, labels_index["TODO Producted & Distributed Energy"],
  11, labels_index["Primary Refined Petroleum Products"], 11, labels_index["Primary Refined Petroleum Products"],
  14, labels_index["Primary Natural Gas"], 14, labels_index["Primary Natural Gas"],
  17, labels_index["TODO Producted & Distributed Energy"], 17, labels_index["TODO Producted & Distributed Energy"],
  20, labels_index["TODO Producted & Distributed Energy"], 20, labels_index["TODO Producted & Distributed Energy"],
  23, labels_index["Primary Nuclear"], 23, labels_index["Primary Nuclear"],
  labels_index["Primary Hydro"], labels_index["Primary Other Renewables and Landfill Gas"],

  # Primary Energy
  labels_index["Electricity"], labels_index["Electricity"], labels_index["Electricity"], labels_index["Electricity"], labels_index["Electricity"], labels_index["Electricity"],

  # Exports
  labels_index["Exports"], labels_index["Exports"], labels_index["Exports"], labels_index["Exports"], labels_index["Exports"], labels_index["Exports"], labels_index["Exports"], labels_index["Exports"],           
]

edge_values = [
  1, 1, 1, 1,
  1, 1, 1, 1,
  1, 1, 1, 1,
  1, 1, 1, 1,
  1, 1, 1, 1,
  1, 1, 1, 1,
  1, 1, 1, 1,
  1, 1, 1, 1,
  1, 1,
  1, 1, 1, 1, 1, 1,
  1, 1, 1, 1, 1, 1, 1, 1,         
]

source_count = len(source_nodes)
target_count = len(target_nodes)
edge_count = len(edge_values)

if source_count != target_count and target_count != edge_values:
  print(f'Source, Target and Edge Values do not match. {source_count}, {target_count}, {edge_count}')

unknown = labels_index["Unknown"]

figure = go.Figure(data = [go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = labels,
      color = "blue"
    ),
    link = dict(
      source = source_nodes,
      target = target_nodes,
      value = edge_values
    )
)])

figure.update_layout(title_text="Basic Sankey Diagram", font_size=10)
figure.show()

In [ ]:
#TODO List:
1. Verify that this is the definitive list of energy produced in Canada
2. Break apart electricity generation and use of energy (layer 2)
3. Electricity generation (layer 3)
4. Sector (layer 4)
5. Detailed sector (layer 5)

5429725000 - 1540083100

SyntaxError: ignored

In [ ]:
end_use_demand_data_2020 = end_use_demand_data[(end_use_demand_data["Year"] == 2020) & (end_use_demand_data["Region"] == "Canada")]
display(end_use_demand_data_2020)

# Production 2019

This section outlines how our energy was produced.

Canada produces *energy* in the following forms:

## Primary Energy
1. Coal
2. Crude Oil
3. Natural Gas
4. Natural Gas liquids (NGL)
5. Hydro
6. Nuclear
7. Steam
8. Wind
9. Solar

## Secondary Energy
8. Coke
9. Coke Oven Gas
10. Refined Petroleum Products
11. Secondary Electricity Sources (i.e. Thermal)

Source: https://www150.statcan.gc.ca/t1/tbl1/en/cv.action?pid=2510003001

See: data/Canada/2510003001-eng.csv

**Note**: All energy values are to be shown as [quads](https://en.wikipedia.org/wiki/Quad_(unit)), where one quad is equal to 10^16 BTU or 1.055 * 10^18 joules (1.055 exajoules)

> Some common types of an energy carrier approximately equal to 1 quad are:

    8,007,000,000 gallons (US) of gasoline
    293,071,000,000 kilowatt-hours (kWh)
    293.07 terawatt-hours (TWh)
    33.434 gigawatt-years (GWy)
    36,000,000 tonnes of coal
    970,434,000,000 cubic feet of natural gas
    5,996,000,000 UK gallons of diesel oil
    25,200,000 tonnes of oil
    252,000,000 tonnes of TNT or five times the energy of the Tsar Bomba nuclear test
    13.3 tonnes of uranium-235


In [ ]:
## Global

production = {}
imports = {}
exports = {}

In [ ]:
## Coal

kiloton_to_tons = 1000
quad_to_kiloton_of_coal = 36000000

def kiloton_of_coal_to_quad(kilotons):
  return kilotons * kiloton_to_tons / quad_to_kiloton_of_coal

production["coal"] = kiloton_of_coal_to_quad(50577.50)
imports["coal"] = kiloton_of_coal_to_quad(0)
exports["coal"] = kiloton_of_coal_to_quad(0)

In [ ]:
## Crude Oil

megalitre_of_crude_oil = 1000000
barrel_of_oil_in_litres = 158.9873
quad_to_boe = 180135869.19434

def megalitre_to_quad(megalitres):
  return megalitres * megalitre_of_crude_oil / barrel_of_oil_in_litres / quad_to_boe

production["crude_oil"] = megalitre_to_quad(272186.3)
imports["crude_oil"] = megalitre_to_quad(45484.7)
exports["crude_oil"] = megalitre_to_quad(219364.7)

In [ ]:
## Natural Gas

# Gigalitre is 1000 * 1000 * 1000 litres. 1000 litres in a cubic metre.
gigalitres_to_cubic_metres = 1000000
# 1 Gm3 NG = 0.035687347874265 quad
cubic_meter_to_quad = 0.035687347874265 / 1000000000

def gigalitre_to_quad(gigalitre):
  return gigalitre * gigalitres_to_cubic_metres * cubic_meter_to_quad

  #TODO: Fill out prod/imports/exports

## Supply 2019

In [ ]:
energy_types = ['Hydro / Wave / Tidal', 'Natural Gas', 'Oil', 'Biomass / Geothermal', 'Coal & Coke', 'Uranium', 'Solar', 'Wind']

# Returns a dictionary with energy production by year in the following format:
# ```
# {
#   "2005": {
#     "Alberta": {
#       "Hydro / Wave / Tidal": value
#     },
#     ...
#   }
# }
# ```
def load_generation_data():
  energy_production = {}
  canada_energy_production = {}

  with open(f'{data_folder}/canada/electricity-generation-2019.csv') as csv_file:
    reader = csv.reader(csv_file)
    # Skip the header line
    next(reader)
  
    for row in reader:
      province = row[1]
      energy_type = row[2]
      year = row[4]
      quantity = float(row[5])

      # We don't want Canada-wide data (we can aggregate it ourselves)
      if province == "Canada":
        year_dictionary = get_nested_dictionary(canada_energy_production, year)
        year_dictionary[energy_type] = quantity
      else:
        year_dictionary = get_nested_dictionary(energy_production, year)
        province_dictionary = get_nested_dictionary(year_dictionary, province)
        province_dictionary[energy_type] = quantity
  return energy_production, canada_energy_production

def get_nested_dictionary(dictionary, key):
  value = dictionary.get(key)
  if value == None:
    dictionary[key] = {}
    value = dictionary[key]
  return value

def extract_energy_values(energy_production, year):
  flattened_values = []
  
  for row in list(energy_production[year].items()):
    energy = [row[0]]
    for energy_type in energy_types:
      energy.append(row[1][energy_type])
    flattened_values.append(energy)

  return np.array(flattened_values)

def style_axes(ax):
  # The following was copied from https://www.pythoncharts.com/matplotlib/grouped-bar-charts-matplotlib/
  ax.spines['top'].set_visible(False)
  ax.spines['right'].set_visible(False)
  ax.spines['left'].set_visible(False)
  ax.spines['bottom'].set_color('#DDDDDD')
  ax.tick_params(bottom=False, left=False)
  ax.set_axisbelow(True)
  ax.yaxis.grid(True, color='#EEEEEE')
  ax.xaxis.grid(False)

  # For each bar in the chart, add a text label so we can easily read the graph
  for bar in ax.patches:
    bar_value = bar.get_height()
    # Don't show any decimal values (they're big enough)
    text = f'{bar_value:.0f}'
    text_x = bar.get_x() + bar.get_width() / 2
    text_y = bar.get_y() + bar_value

    ax.text(text_x, text_y, text, ha='center', va='bottom', color=bar.get_facecolor(), size=12)

def plot_energy_by_province(title, energy_values):
  province_names = energy_values[:, 0]
  province_count = len(province_names)
  energy_type_count = len(energy_types)

  bar_width = 1.0
  full_width = energy_type_count * bar_width
  half_width = full_width / 2
  padding = bar_width * 2

  # This represents the location of the xticks (or the center of the 'bars') so we need to account for the full width of the bars, otherwise we overlap
  x = np.arange(province_count) * (full_width + padding)

  fig_width = bar_width * (province_count) * energy_type_count

  fig, ax = plt.subplots(figsize=(fig_width, 10))
  for i in range(energy_type_count):
    ax.bar(x - half_width + i * bar_width, energy_values[:, i + 1].astype(np.float), width=bar_width)

  style_axes(ax)

  plt.xticks(x - bar_width / 2, province_names)
  plt.ylabel('GWh Generated')
  plt.xlabel('Provinces')
  plt.title(title)
  plt.legend(energy_types)

  plt.show()

In [ ]:
# Plot 2019 Data

energy_production, canada_energy_production = load_generation_data()
energy_values = extract_energy_values(energy_production, '2019')
plot_energy_by_province('2019 Energy Production - Canada', energy_values)

## Demand 2019


## Emissions


## Solutions